In [2]:
import socket
from threading import Thread
import pickle
import time 
from util import *

BLOCKCHAIN = {}
CLOCK = LamportClock(0,0)
CONNECTIONS = {}
PID = 0

In [ ]:
class Connections(Thread):
    def __init__(self,connection):
        Thread.__init__(self)
        self.connection = connection

    def run(self):
        # global replyCount
        # global requestPriorityQueue
        # global transactionFlag
        global CLOCK

        while True:
            response = self.connection.recv(buff_size)
            data = pickle.loads(response)
            CLOCK.updateClock(data.clock)
            print("Current clock of process " + str(PID) + " is " + str(CLOCK))

            if data.reqType == "MUTEX":
                requestPriorityQueue.append(LamportClock(data.reqClock.clock, data.reqClock.pid))
                heapq.heapify(requestPriorityQueue)
                print("REQUEST recieved from " + str(data.fromPid) + " at " + str(CLOCK))
                CLOCK.incrementClock()
                sleep()
                print("REPLY sent to " + str(data.fromPid) + " at " + str(CLOCK))
                print("Current clock of process " + str(pid) + " is " + str(CLOCK))
                reply = RequestMessage(PID, CLOCK, "REPLY")
                self.connection.send(pickle.dumps(reply))

            if data.reqType == "REPLY":
                print("REPLY recieved from " + str(data.fromPid) + " at " + str(CLOCK))
                replyCount += 1
                if replyCount == 2 and requestPriorityQueue[0].pid == PID:
                    print("Local Queue:")
                    for i in requestPriorityQueue:
                        print(str(i))
                    self.handle_transaction(data)
                    heapq.heappop(requestPriorityQueue)
                    heapq.heapify(requestPriorityQueue)
                    replyCount = 0
                    broadcast("RELEASE")
                    transactionFlag = True

#             if data.reqType == "RELEASE":
#                 #print("Inside release")
#                 print("Local Queue:")
#                 for i in requestPriorityQueue:
#                     print(str(i))
#                 print("RELEASE recieved from " + str(data.fromPid) + " at " + str(myClock))
#                 heapq.heappop(requestPriorityQueue)
#                 heapq.heapify(requestPriorityQueue)
#                 if len(requestPriorityQueue) > 0 and requestPriorityQueue[0].pid == pid and replyCount == 2:
#                     #print("Execute Transaction")
#                     self.handle_transaction(data)
#                     heapq.heappop(requestPriorityQueue)
#                     heapq.heapify(requestPriorityQueue)
#                     replyCount = 0
#                     #release = RequestMessage(pid, myClock, "RELEASE")
#                     #myClock.incrementClock()
#                     #print("Current clock of process " + str(pid) + " is " + str(myClock))
#                     #print("Sending RELEASE to all clients")
#                     broadcast("RELEASE")
#                     transactionFlag = True

#     def handle_transaction(self, data):
#         if user_input == "BAL":
#             request = RequestMessage(pid, myClock,"BALANCE")
#             myClock.incrementClock()
#             sleep()
#             print("Balance request sent to server" + " at " + str(myClock))
#             #print("Current clock of process " + str(pid) + " is " + str(myClock))
#             c2c_connections[0].sendall(pickle.dumps(request))
#             #print("balance at 91")

#             balance = c2c_connections[0].recv(buff_size)
#             myClock.incrementClock()
#             #print("Current clock of process " + str(pid) + " is " + str(myClock))
#             sleep()
#             print("=============================")
#             print("Balance is " + str(balance))
#             print("=============================")
#             #transactionFlag = True
#         else:
#             reciever, amount = user_input.split()

#             transaction = Transaction(int(pid), int(reciever), int(amount))

#             myClock.incrementClock()
#             #print("Current clock of process " + str(pid) + " is " + str(myClock))
#             sleep()
#             print("Balance request sent to server" + " at " + str(myClock))
#             request = RequestMessage(pid,myClock,"BALANCE")
#             c2c_connections[0].sendall(pickle.dumps(request))
#             balance = c2c_connections[0].recv(buff_size)
#             myClock.incrementClock()
#             #print("Current clock of process " + str(pid) + " is " + str(myClock))
#             sleep()
#             print("======================================")
#             print("Balance before transaction " + str(balance))

#             if int(balance) >= int(amount):

#                 myClock.incrementClock()
#                 #print("Current clock of process " + str(pid) + " is " + str(myClock))
#                 request = RequestMessage(pid,myClock,"LAST_BLOCK")
#                 c2c_connections[0].send(pickle.dumps(request))
#                 last_blck_str = c2c_connections[0].recv(buff_size)

#                 myClock.incrementClock()
#                 #print("Current clock of process " + str(pid) + " is " + str(myClock))
#                 last_blck = pickle.loads(last_blck_str)
#                 block = Block(hashlib.sha256(str(last_blck)).digest(), 
#                     Transaction(transaction.sender,transaction.reciever,transaction.amount))
#                 request = RequestMessage(pid,data.clock,"ADD_BLOCK", None, block)

#                 myClock.incrementClock()
#                 #print("Current clock of process " + str(pid) + " is " + str(myClock))
#                 c2c_connections[0].send(pickle.dumps(request))

#                 message = c2c_connections[0].recv(buff_size)
#                 myClock.incrementClock()
#                 #print("Current clock of process " + str(pid) + " is " + str(myClock))
#                 print("Transaction was " + str(message))
#                 print("Balance after transaction " + str(int(balance) - int(amount)))
#                 print("======================================")
#             else:
#                 print("Insufficient Balance")
#                 print("======================================")
#             #transactionFlag = True

def sleep():
    time.sleep(SLEEP_TIME)

def sendRequest(client, reqType, reqClock):
    CLOCK.incrementClock()
    print("Current clock of process " + str(pid) + " is " + str(CLOCK))
    sleep()
    if reqType == "MUTEX":
        print("REQUEST sent to " + str(client) + " at " + str(CLOCK))
    elif reqType == "RELEASE":
        print("RELEASE sent to " + str(client) + " at " + str(CLOCK))

    msg = RequestMessage(PID, CLOCK, reqType, reqClock)
    data_string = pickle.dumps(msg)
    CONNECTIONS[client].sendall(data_string)

def broadcast(reqType, reqClock = None):
    for dest in range(1, CLIENT_COUNT+1):
        if PID != dest:
            sendRequest(dest, reqType, reqClock)

In [ ]:
def close_sockets():
    for connection in CONNECTIONS.values():
        connection.close()

def get_connection(source, dest):
    client2client = socket.socket()
    client2client.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    client2client.bind((ip, CLIENT_TO_CLIENT_PORTS[source][dest]))
    if dest > source:
        # Accept the connection from the dest server
        client2client.listen(1)
        conn, client_address = client2client.accept()
        print("Connected to Client {} at port {}".format(dest, client_address[1]))
        CONNECTIONS[dest] = conn
        new_client = Connections(conn)
        new_client.start()
    else:
        try:
            client2client.connect((HOST, CLIENT_TO_CLIENT_PORTS[dest][source]))
            print("Connected to Client {} at port {}".format(dest, CLIENT_TO_CLIENT_PORTS[dest][source]))
        except socket.error as e:
            print(str(e))

def main():
    if int(sys.argv[1])>CLIENT_COUNT or int(sys.argv[1])<0:
        print("PID not in the set of allowed pids".format(sys.argv[1]))
        exit()
    PID = int(sys.argv[1])
    current_port = CLIENT_TO_SERVER_PORTS[PID]
    
    # Connection to the server
    print("Initiating connection to the server")
    # Bind to the current port
    client_socket = socket.socket()
    try:
        client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        client_socket.bind((HOST, current_port))
        print("Hosted the client for server at {}".format(current_port))
    except socket.error as e:
        print(str(e))
        exit()
    CLOCK = LamportClock(0, current_pid)
    # Connect to the server
    try:
        client_socket.connect((HOST, SERVER_PORT))
        print("Connected to server")
    except socket.error as e:
        print(str(e))
        exit()
    
    CONNECTIONS[0] = client_socket
    
    # Connection to the clients
    for dest in range(1, CLIENT_COUNT+1):
        if PID != dest:
            get_connection(PID, dest)
    
    print("=======================================================")
    print("Current Balance is $10")
    
    while True:
        transactionFlag = False
        print("=======================================================")
        print("| For Balance type : 'BAL'                              |")
        print("| For transferring money type : 'RECV_ID AMOUNT' Eg.(2 5)    |")
        print("| To quit type 'Q'                                    |") 
        print("=======================================================")
        user_input = raw_input()

        if user_input != "Q" and user_input != "BAL" and len(user_input.split()) != 2:
            print("Please enter valid input")
            continue

        if user_input == "Q":
            break

        CLOCK.incrementClock()
        print("Current clock of process " + str(pid) + " : " + str(myClock))
        requestClock = LamportClock(CLOCK.clock, CLOCK.pid)
        # requestPriorityQueue.append(requestClock)
        # heapq.heapify(requestPriorityQueue)
        # replyCount = 0

        # broadcast("MUTEX", requestClock)

        # while transactionFlag == False:
            # a=1

    close_sockets()

if __name__ == "__main__":
    main()

In [3]:
RequestMessage(2, LamportClock(0, 0), "BAL)

util.RequestMessage

In [7]:
type(pickle.loads(pickle.dumps(20)))

int

In [8]:
a, b = [1, 2]
a

1

In [9]:
b

2

In [16]:
import hashlib
hashlib.sha256(pickle.dumps("")).digest()

b'\x8e*\x8b\xd9\x96\xc2\x9b\x99\xc9\tfN\xe5\x1e\xb5\xee;#X0jBi\xd5\xb6\x9d\xd9\x90u\xe6\xdd\x85'

In [13]:
hashlib.sha256?

Signature: hashlib.sha256(string=b'', *, usedforsecurity=True)
Docstring: Returns a sha256 hash object; optionally initialized with a string
Type:      builtin_function_or_method


In [17]:
list.insert

Signature: list.insert(self, index, object, /)
Docstring: Insert object before index.
Type:      method_descriptor


In [25]:
x = [1, 2, 3, 4]
x.insert(2, 20)
x

[1, 2, 20, 3, 4]

In [26]:
x.insert(5, 50)

In [27]:
x

[1, 2, 20, 3, 4, 50]

In [28]:
x.insert(7, 80)
x

[1, 2, 20, 3, 4, 50, 80]

In [30]:
x.insert(0, 100)
x

[100, 1, 2, 20, 3, 4, 50, 100, 80]

In [31]:
for idx in range(3, -1, -1):
    print(idx)

3
2
1
0


In [32]:
for idx in range(3, 3):
    print(idx)

In [39]:
b"\xe3\xb0\xc4B\x98\xfc\x1c\x14\x9a\xfb\xf4\xc8\x99o\xb9$'\xaeA\xe4d\x9b\x93L\xa4\x95\x99\x1bxR\xb8U"

b"\xe3\xb0\xc4B\x98\xfc\x1c\x14\x9a\xfb\xf4\xc8\x99o\xb9$'\xaeA\xe4d\x9b\x93L\xa4\x95\x99\x1bxR\xb8U"

In [40]:
1 + \
2

3

In [43]:
LamportClock(2, 1).

AttributeError: 'LamportClock' object has no attribute 'copy'